<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [268]:
##### Your Code Here #####
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
# fix random seed for reproducibility
numpy.random.seed(42)
seed = 42

In [246]:
# load dataset
from sklearn.datasets import load_boston
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn - WA_Fn-UseC_-Telco-Customer-Churn.csv')

print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [247]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [248]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [249]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,8984-HPEMB,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174


In [250]:
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].mean())

In [251]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

In [252]:
categorical = df.describe(exclude='number').columns
numerical = df.describe().columns[:-1]


In [253]:
df[categorical] = enc.fit_transform(df[categorical])

In [254]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375.0,0.0,0,1.0,0.0,1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,29.85,29.85,0.0
1,3962.0,1.0,0,0.0,0.0,34,1.0,0.0,0.0,2.0,...,2.0,0.0,0.0,0.0,1.0,0.0,3.0,56.95,1889.50,0.0
2,2564.0,1.0,0,0.0,0.0,2,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,53.85,108.15,1.0
3,5535.0,1.0,0,0.0,0.0,45,0.0,1.0,0.0,2.0,...,2.0,2.0,0.0,0.0,1.0,0.0,0.0,42.30,1840.75,0.0
4,6511.0,0.0,0,0.0,0.0,2,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,70.70,151.65,1.0


In [255]:
df['MonthlyCharges'].values

array([ 29.85,  56.95,  53.85, ...,  29.6 ,  74.4 , 105.65])

In [213]:
scaler = StandardScaler()


In [257]:

# Split into X and y and turn into numpy arays
y = df.Churn.values


X = df.drop(["Churn","customerID"], axis='columns').values

# Scale input data
#scaler = StandardScaler()
#X = scaler.fit_transform(X)
#print(X)

In [258]:
X[0][0]

0.0

In [259]:
from keras.utils import to_categorical

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375.0,0.0,0,1.0,0.0,1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,29.85,29.85,0.0
1,3962.0,1.0,0,0.0,0.0,34,1.0,0.0,0.0,2.0,...,2.0,0.0,0.0,0.0,1.0,0.0,3.0,56.95,1889.50,0.0
2,2564.0,1.0,0,0.0,0.0,2,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,53.85,108.15,1.0
3,5535.0,1.0,0,0.0,0.0,45,0.0,1.0,0.0,2.0,...,2.0,2.0,0.0,0.0,1.0,0.0,0.0,42.30,1840.75,0.0
4,6511.0,0.0,0,0.0,0.0,2,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,70.70,151.65,1.0


In [260]:
df.describe()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,...,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,0.504756,0.162147,0.483033,0.299588,32.371149,0.903166,0.940508,0.872923,0.790004,...,0.904444,0.797104,0.985376,0.992475,0.690473,0.592219,1.574329,64.761692,2283.300441,0.265370
std,2033.283305,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.948554,0.737796,0.859848,...,0.879949,0.861551,0.885002,0.885091,0.833755,0.491457,1.068104,30.090047,2265.000258,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000,0.000000
25%,1760.500000,0.000000,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,35.500000,402.225000,0.000000
50%,3521.000000,1.000000,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,70.350000,1400.550000,0.000000
75%,5281.500000,1.000000,0.000000,1.000000,1.000000,55.000000,1.000000,2.000000,1.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,89.850000,3786.600000,1.000000
max,7042.000000,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,3.000000,118.750000,8684.800000,1.000000


In [261]:
X.shape[1], y.shape

(19, (7043,))

In [262]:
df.Churn.unique()

array([0., 1.])

In [238]:
# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 32

# Create Model
model = Sequential()
model.add(Dense(16, input_dim=inputs, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# Fit Model
model.fit(X, y, validation_split=0.33, epochs=10, batch_size=batch_size)

Train on 4718 samples, validate on 2325 samples
Epoch 1/10
4718/4718 [==============================] - 1s 237us/step - loss: 2.6557 - acc: 0.7062 - val_loss: 1.0625 - val_acc: 0.7548
Epoch 2/10
4718/4718 [==============================] - 0s 40us/step - loss: 0.7504 - acc: 0.7399 - val_loss: 0.4810 - val_acc: 0.7716
Epoch 3/10
4718/4718 [==============================] - 0s 49us/step - loss: 0.6785 - acc: 0.7569 - val_loss: 1.1123 - val_acc: 0.7849
Epoch 4/10
4718/4718 [==============================] - 0s 41us/step - loss: 0.7808 - acc: 0.7476 - val_loss: 0.5223 - val_acc: 0.7837
Epoch 5/10
4718/4718 [==============================] - 0s 37us/step - loss: 0.5633 - acc: 0.7698 - val_loss: 0.4799 - val_acc: 0.7544
Epoch 6/10
4718/4718 [==============================] - 0s 39us/step - loss: 0.5292 - acc: 0.7738 - val_loss: 0.7365 - val_acc: 0.7789
Epoch 7/10
4718/4718 [==============================] - 0s 39us/step - loss: 0.5688 - acc: 0.7683 - val_loss: 1.1943 - val_acc: 0.7763
Epoch 

In [240]:
model.predict(X).min()

0.0

In [263]:

# Split into X and y and turn into numpy arays
y = df.Churn.values


X = df.drop(["Churn","customerID"], axis='columns').values

# Scale input data
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [287]:
# Important Hyperparameters
inputs = X.shape[1]
epochs = 50
batch_size = 32

# Create Model
model = Sequential()
model.add(Dense(38, input_dim=inputs, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# Fit Model
model.fit(X, y, validation_split=0.33, epochs=20, batch_size=batch_size)

Train on 4718 samples, validate on 2325 samples
Epoch 1/20
4718/4718 [==============================] - 3s 663us/step - loss: 0.5353 - acc: 0.7147 - val_loss: 0.4666 - val_acc: 0.7763
Epoch 2/20
4718/4718 [==============================] - 0s 69us/step - loss: 0.4336 - acc: 0.7836 - val_loss: 0.4499 - val_acc: 0.7910
Epoch 3/20
4718/4718 [==============================] - 0s 63us/step - loss: 0.4190 - acc: 0.7959 - val_loss: 0.4454 - val_acc: 0.7987
Epoch 4/20
4718/4718 [==============================] - 0s 62us/step - loss: 0.4110 - acc: 0.7993 - val_loss: 0.4443 - val_acc: 0.7983
Epoch 5/20
4718/4718 [==============================] - 0s 63us/step - loss: 0.4056 - acc: 0.8048 - val_loss: 0.4396 - val_acc: 0.7966
Epoch 6/20
4718/4718 [==============================] - 0s 65us/step - loss: 0.4030 - acc: 0.8063 - val_loss: 0.4405 - val_acc: 0.8026
Epoch 7/20
4718/4718 [==============================] - 0s 67us/step - loss: 0.3987 - acc: 0.8124 - val_loss: 0.4399 - val_acc: 0.8069
Epoch 

In [296]:
# Keras imports
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
# sklearn imports
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Important Hyperparameters
inputs = X.shape[1]
epochs = 10
batch_size = 32

# define base model
def baseline_model():
  # create model
    model = Sequential()
    model.add(Dense(38, input_dim=inputs, activation='relu'))
    model.add(Dense(19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=epochs, batch_size=batch_size,
    verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(pipeline, X, y, cv=kfold)
print(f"K-fold Cross-Val Results - Mean: {-results.mean():.2f} StDev: {results.std():.2f} MSE")

K-fold Cross-Val Results - Mean: 0.43 StDev: 0.01 MSE


In [284]:
results

array([-0.41195969, -0.40454668, -0.44016466, -0.45338183, -0.43408336])

In [291]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

# fix random seed for reproducibility
seed = 42
numpy.random.seed(seed)

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, y):
  # create model
  model = Sequential()
  model.add(Dense(38, input_dim=19, activation='relu'))
  model.add(Dense(19, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Fit the model
  model.fit(X[train], y[train], epochs=10, batch_size=10, verbose=0)
  # evaluate the model
  scores = model.evaluate(X[test], y[test], verbose=0)
  print(f'{model.metrics_names[1]}: {(scores[1]*100):.2f}%') 
  cvscores.append(scores[1]*100)
print(f'{numpy.mean(cvscores):.2f}% +/- {numpy.std(cvscores):.2f}%')

acc: 80.41%
acc: 80.48%
acc: 78.35%
acc: 79.42%
acc: 79.74%
79.68% +/- 0.78%


In [297]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
  # create model
        model = Sequential()
        model.add(Dense(38, input_dim=19, activation='relu'))
        model.add(Dense(19, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
  # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

C:\Users\mhdal\Anaconda3\envs\lambda-neural\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.8002271784243894 using {'batch_size': 60, 'epochs': 20}
Means: 0.7961096100029952, Stdev: 0.0013647352184608407 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7905722032736838, Stdev: 0.006227957571291364 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7978134327410095, Stdev: 0.0004931025843609994 with: {'batch_size': 40, 'epochs': 20}
Means: 0.8002271784243894, Stdev: 0.001532294901941202 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7990912961364046, Stdev: 0.0012252929240128077 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7971035080607364, Stdev: 0.0031924955226660494 with: {'batch_size': 100, 'epochs': 20}


In [ ]:
# define the grid search parameters
param_grid = {'batch_size': [60],
              'epochs': [20, 40, 60]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [304]:
# define the grid search parameters
param_grid = {'batch_size': [60],
              'epochs': [20]}
scores=[]
# Function to create model, required for KerasClassifier
for opt in ['Adadelta','Adagrad','Adam','Adamax','NAdam','RMSprop','SGD']:

    model = Sequential()
    model.add(Dense(38, input_dim=19, activation='relu'))
    model.add(Dense(19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
# Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    # Fit Model
    model.fit(X, y, validation_split=0.33, epochs=20, batch_size=batch_size, verbose=0)
    print(opt, model.evaluate(X[test], y[test], verbose=0))


Adadelta [0.39305952068500344, 0.8159203974592322]
Adagrad [0.41404993367178183, 0.7960198999467943]
Adam [0.39684778532409126, 0.8137882012971852]
Adamax [0.4018260361289165, 0.810234541662551]
NAdam [0.38877962325258125, 0.8187633256752946]
RMSprop [0.40439847713255594, 0.8059701493384572]
SGD [0.4266160499481928, 0.7953091678927787]


In [301]:
print('done')

done


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?